# SESION 4 DICIEMBRE

# Modificamos columnas que eran object a float 

In [1]:
import pandas as pd
import numpy as np
# -----------------------------------------------------------------------
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer

# Librerías de visualización
# -----------------------------------------------------------------------
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:
df_rawdata = pd.read_csv("../Data/raw_data_20231201.csv", index_col=0)

In [4]:
def cambiar_comas(cadena):
    try:
        return float(cadena.replace(",", "."))
    except:
        return np.nan
    
df_rawdata["Monthlyincome"] = df_rawdata["Monthlyincome"].apply(cambiar_comas)
df_rawdata["Performancerating"] = df_rawdata["Performancerating"].apply(cambiar_comas)
df_rawdata["Totalworkingyears"] = df_rawdata["Totalworkingyears"].apply(cambiar_comas)

In [5]:
df_rawdata["Employeenumber"]= df_rawdata["Employeenumber"].apply(cambiar_comas)

In [6]:
df_rawdata["Employeenumber"].isnull().sum()

431

# Gestión de nulos

Cambiamos los nulos de Employeenumber con un "auto-incremental"

In [7]:
# Encuentra el último ID existente
ultimo_id = df_rawdata['Employeenumber'].max()
# Define una variable para el siguiente ID después del último
siguiente_id = ultimo_id + 1
# Itera sobre los índices del DataFrame
for indice in df_rawdata.index:
    if pd.isnull(df_rawdata.at[indice, 'Employeenumber']):  # Verifica si el valor es nulo
        df_rawdata.at[indice, 'Employeenumber'] = siguiente_id  # Asigna el siguiente ID
        siguiente_id += 1  # Incrementa el siguiente ID
df_rawdata["Employeenumber"] = df_rawdata["Employeenumber"].astype(int)
# Muestra el DataFrame actualizado
print(df_rawdata["Employeenumber"].tail(10))

1604     927
1605     941
1606     956
1607    2497
1608     966
1609     967
1610     972
1611    2498
1612     990
1613    2499
Name: Employeenumber, dtype: int64


In [8]:
# Muestra el DataFrame actualizado
df_rawdata["Employeenumber"].dtype

dtype('int64')

In [9]:
nulls_cat = df_rawdata[df_rawdata.columns[df_rawdata.isnull().any()]].select_dtypes(include = "O").columns
nulls_cat

Index(['Businesstravel', 'Department', 'Educationfield', 'Maritalstatus',
       'Overtime'],
      dtype='object')

In [10]:
(df_rawdata[['Businesstravel', 'Department', 'Educationfield', 'Maritalstatus','Overtime']].isnull().sum() / df_rawdata.shape[0]) * 100

Businesstravel    47.880795
Department        81.258278
Educationfield    46.490066
Maritalstatus     40.264901
Overtime          41.721854
dtype: float64

In [11]:
# sacamos el 'value_counts()' de cada una de las columnas categóricas que tienen nulos para saber como es la distribución de sus categorías
for col in nulls_cat:
    print(f"La distribución de las categorías para la columna {col.upper()}")
    display(df_rawdata[col].value_counts() / df_rawdata.shape[0])
    print("........................")

La distribución de las categorías para la columna BUSINESSTRAVEL


Businesstravel
travel_rarely        0.364901
travel_frequently    0.101325
non-travel           0.054967
Name: count, dtype: float64

........................
La distribución de las categorías para la columna DEPARTMENT


Department
 Research & Development     0.122517
 Sales                      0.055629
 Human Resources            0.009272
Name: count, dtype: float64

........................
La distribución de las categorías para la columna EDUCATIONFIELD


...

........................
La distribución de las categorías para la columna MARITALSTATUS


Maritalstatus
Married     0.270861
Single      0.202649
Divorced    0.123841
Name: count, dtype: float64

........................
La distribución de las categorías para la columna OVERTIME


Overtime
No     0.422517
Yes    0.160265
Name: count, dtype: float64

........................


In [12]:
nulls_cat = ['Businesstravel', 'Educationfield', 'Maritalstatus', 'Overtime']

Todos desconocidos!

In [13]:
for columna in nulls_cat:
    # reemplazamos los nulos por el valor Unknown para cada una de las columnas de la lista
    df_rawdata[columna] = df_rawdata[columna].fillna("Unknown")

In [14]:
df_rawdata["Overtime"].unique()

array(['No', 'Unknown', 'Yes'], dtype=object)

# Columna Department: vinculada con Jobrole

In [15]:
# primero vemos cómo está la columna Jobrole
#df_rawdata["Jobrole"].head()

In [16]:
# # capitalize valores de Jobrole (primera letra de cada palabra)
# df_rawdata['Jobrole'] = df_rawdata['Jobrole'].apply(lambda x: ' '.join(word.capitalize() for word in x.lower().split()))
# df_rawdata["Jobrole"].head()

In [17]:
# hacemos función para asignar los valores nulos en Department según la columna Jobrole
def fill_department(df, jobrole, department):
    df.loc[df['Jobrole'] == jobrole, 'Department'] = df.loc[df['Jobrole'] == jobrole, 'Department'].fillna(department)

# Uso de la función fill_department para reemplazar los valores faltantes en 'Department'
fill_department(df_rawdata, 'Research Director', 'Research & Development')
fill_department(df_rawdata, 'Healthcare Representative', 'Research & Development')
fill_department(df_rawdata, 'Laboratory Technician', 'Research & Development')
fill_department(df_rawdata, 'Manufacturing Director', 'Research & Development')
fill_department(df_rawdata, 'Research Scientist', 'Research & Development')
fill_department(df_rawdata, 'Sales Executive', 'Sales')
fill_department(df_rawdata, 'Sales Representative', 'Sales')
fill_department(df_rawdata, 'Human Resources', 'Human Resources')
fill_department(df_rawdata, 'Manager', 'Unknown')

df_rawdata["Department"] = df_rawdata["Department"].str.strip()
# Mostrar el DataFrame actualizado
df_rawdata[["Jobrole", "Department"]].head(5)

,Jobrole,Department
0,Research Director,NaN
1,Manager,NaN
2,Manager,Research & Development
3,Research Director,NaN
4,Sales Executive,NaN


In [18]:
# Mostrar estadísticas descriptivas y conteo de valores únicos
print("Estadísticas descriptivas para 'Jobrole':\n", df_rawdata['Jobrole'].describe())
print("\nEstadísticas descriptivas para 'Department':\n", df_rawdata['Department'].describe())

print("\nConteo de valores únicos para 'Jobrole':\n", df_rawdata['Jobrole'].value_counts())
print("\nConteo de valores únicos para 'Department':\n", df_rawdata['Department'].value_counts())

Estadísticas descriptivas para 'Jobrole':
 count                  1510
unique                    9
top        Sales Executive 
freq                    336
Name: Jobrole, dtype: object

Estadísticas descriptivas para 'Department':
 count                        283
unique                         3
top       Research & Development
freq                         185
Name: Department, dtype: object

Conteo de valores únicos para 'Jobrole':
 Jobrole
 Sales Executive               336
 Research Scientist            300
 Laboratory Technician         264
 Manufacturing Director        148
 Healthcare Representative     137
 Manager                       105
 Sales Representative           84
 Research Director              83
 Human Resources                53
Name: count, dtype: int64

Conteo de valores únicos para 'Department':
 Department
Research & Development    185
Sales                      84
Human Resources            14
Name: count, dtype: int64


# Todas las columnas categóricas listas 

In [19]:
# Como hay valores negativos en la columna Distancefromhome los cambiamos a nulos.
def negative_to_null(data):
    if data < 0:
        return np.nan
    else:
        return data
    
df_rawdata["Distancefromhome"] = df_rawdata["Distancefromhome"].apply(negative_to_null)

In [20]:
df_rawdata["Distancefromhome"].tail(5)

1609    3.0
1610    4.0
1611    NaN
1612    8.0
1613    7.0
Name: Distancefromhome, dtype: float64

## Gestión nulos columnas numéricas

In [21]:
nulls_num = df_rawdata[df_rawdata.columns[df_rawdata.isna().any()]].columns
nulls_num

Index(['Dailyrate', 'Department', 'Distancefromhome', 'Monthlyincome',
       'Performancerating', 'Totalworkingyears', 'Worklifebalance'],
      dtype='object')

In [22]:
# Función para quitar "Not Available" y poner nulos en su lugar
def not_available_to_null(data):
    if data == 'Not Available':
        return np.nan
    else:
        return float(data)
df_rawdata["Hourlyrate"] = df_rawdata["Hourlyrate"].apply(not_available_to_null)

In [23]:
df_rawdata_copy = df_rawdata.copy()

In [24]:
print(f"Tenemos nulos: \n{df_rawdata_copy[['Dailyrate', 'Distancefromhome', 'Monthlyincome', 'Performancerating','Totalworkingyears', 'Worklifebalance', 'Hourlyrate']].isnull().sum()} nulos")			

Tenemos nulos: 
Dailyrate            116
Distancefromhome     178
Monthlyincome        799
Performancerating    182
Totalworkingyears    494
Worklifebalance      100
Hourlyrate            80
dtype: int64 nulos


# IterativeImputer

In [25]:
# instanciamos las clases										
imputer_iterative = IterativeImputer(max_iter = 20, random_state = 42)			
# ajustamos y tranformamos los datos										
imputer_iterative_imputado = imputer_iterative.fit_transform(df_rawdata_copy[['Dailyrate', 'Distancefromhome', 'Monthlyincome', 'Performancerating','Totalworkingyears', 'Worklifebalance', 'Hourlyrate']])
# comprobamos que es lo que nos devuelve, que en este caso es un array también					
imputer_iterative_imputado																						

array([[6.84000000e+02, 6.00000000e+00, 1.95370000e+04, ...,
        3.26175277e+01, 3.00000000e+00, 5.10000000e+01],
       [6.99000000e+02, 1.00000000e+00, 1.99990000e+04, ...,
        3.40000000e+01, 3.00000000e+00, 6.50000000e+01],
       [5.32000000e+02, 4.00000000e+00, 1.92320000e+04, ...,
        2.20000000e+01, 2.90522101e+00, 5.80000000e+01],
       ...,
       [9.03000000e+02, 9.05859584e+00, 4.96921235e+03, ...,
        9.00000000e+00, 3.00000000e+00, 4.10000000e+01],
       [1.22900000e+03, 8.00000000e+00, 6.38325083e+03, ...,
        1.20000000e+01, 3.00000000e+00, 8.40000000e+01],
       [5.66000000e+02, 7.00000000e+00, 1.08450000e+04, ...,
        1.84083797e+01, 3.00000000e+00, 7.50000000e+01]])

In [26]:
df_rawdata_copy[['Dailyrate_ITE', 'Distancefromhome_ITE', 'Monthlyincome_ITE', 'Performancerating_ITE','Totalworkingyears_ITE', 'Worklifebalance_ITE', 'Hourlyrate_ITE']] = imputer_iterative_imputado										
										
# comprobamos los nulos										
print(f"Después del 'Iterative' tenemos: \n{df_rawdata_copy[['Dailyrate_ITE', 'Distancefromhome_ITE', 'Monthlyincome_ITE', 'Performancerating_ITE', 'Totalworkingyears_ITE', 'Worklifebalance_ITE', 'Hourlyrate_ITE']].isnull().sum()} nulos")										
										

Después del 'Iterative' tenemos: 
Dailyrate_ITE            0
Distancefromhome_ITE     0
Monthlyincome_ITE        0
Performancerating_ITE    0
Totalworkingyears_ITE    0
Worklifebalance_ITE      0
Hourlyrate_ITE           0
dtype: int64 nulos


# KNN Imputer

In [27]:
# instanciamos la clase del KNNImputer
imputer_knn = KNNImputer(n_neighbors = 5)
# ajustamos y transformamos los datos
imputer_knn_imputado = imputer_knn.fit_transform(df_rawdata_copy[['Dailyrate', 'Distancefromhome', 'Monthlyincome', 'Performancerating','Totalworkingyears', 'Worklifebalance', 'Hourlyrate']])
# comprobamos que es lo que nos devuelve, que sigue siendo un array
imputer_knn_imputado

array([[6.8400e+02, 6.0000e+00, 1.9537e+04, ..., 1.3600e+01, 3.0000e+00,
        5.1000e+01],
       [6.9900e+02, 1.0000e+00, 1.9999e+04, ..., 3.4000e+01, 3.0000e+00,
        6.5000e+01],
       [5.3200e+02, 4.0000e+00, 1.9232e+04, ..., 2.2000e+01, 3.0000e+00,
        5.8000e+01],
       ...,
       [9.0300e+02, 1.0600e+01, 7.1448e+03, ..., 9.0000e+00, 3.0000e+00,
        4.1000e+01],
       [1.2290e+03, 8.0000e+00, 5.8398e+03, ..., 1.2000e+01, 3.0000e+00,
        8.4000e+01],
       [5.6600e+02, 7.0000e+00, 1.0845e+04, ..., 1.5000e+01, 3.0000e+00,
        7.5000e+01]])

In [28]:
# por último nos queda añadir ese array al DataFrame como hemos hecho hasta ahora
df_rawdata_copy[['Dailyrate_KNN', 'Distancefromhome_KNN', 'Monthlyincome_KNN', 'Performancerating_KNN','Totalworkingyears_KNN', 'Worklifebalance_KNN', 'Hourlyrate_KNN']] = imputer_knn_imputado

# comprobamos los nulos
print(f"Después del 'KNN' tenemos: \n{df_rawdata_copy[['Dailyrate_KNN', 'Distancefromhome_KNN', 'Monthlyincome_KNN', 'Performancerating_KNN','Totalworkingyears_KNN', 'Worklifebalance_KNN', 'Hourlyrate_KNN']].isnull().sum()} nulos")


Después del 'KNN' tenemos: 
Dailyrate_KNN            0
Distancefromhome_KNN     0
Monthlyincome_KNN        0
Performancerating_KNN    0
Totalworkingyears_KNN    0
Worklifebalance_KNN      0
Hourlyrate_KNN           0
dtype: int64 nulos


In [29]:
df_rawdata_copy.describe()[['Dailyrate', 'Dailyrate_ITE', 'Dailyrate_KNN']]

,Dailyrate,Dailyrate_ITE,Dailyrate_KNN
count,1394.000000,1510.000000,1510.000000
mean,807.050215,807.859002,811.300662
std,401.842182,386.577732,388.916129
min,103.000000,103.000000,103.000000
25%,472.500000,504.250000,501.750000
50%,805.500000,807.451017,814.000000
75%,1157.750000,1135.500000,1137.750000
max,1499.000000,1499.000000,1499.000000


In [30]:
df_rawdata_copy.describe()[["Distancefromhome", "Distancefromhome_ITE", "Distancefromhome_KNN"]]

,Distancefromhome,Distancefromhome_ITE,Distancefromhome_KNN
count,1332.000000,1510.000000,1510.000000
mean,8.960961,8.964555,9.123974
std,7.954886,7.471860,7.571923
min,1.000000,1.000000,1.000000
25%,2.000000,2.000000,2.000000
50%,7.000000,8.000000,7.700000
75%,13.000000,11.000000,13.000000
max,29.000000,29.000000,29.000000


In [31]:
df_rawdata_copy.describe()[["Monthlyincome", "Monthlyincome_ITE", "Monthlyincome_KNN"]]

,Monthlyincome,Monthlyincome_ITE,Monthlyincome_KNN
count,711.000000,1510.000000,1510.000000
mean,6527.918425,6379.350669,6210.313775
std,4810.541121,4045.555183,3600.306711
min,1009.000000,-440.442943,1009.000000
25%,2906.500000,3527.203958,3958.150000
50%,4850.000000,5477.298448,5417.000000
75%,8456.000000,7423.212705,7260.200000
max,19999.000000,21083.516020,19999.000000


In [32]:
df_rawdata_copy.describe()[["Performancerating", "Performancerating_ITE", "Performancerating_KNN"]]

,Performancerating,Performancerating_ITE,Performancerating_KNN
count,1328.000000,1510.000000,1510.000000
mean,3.155120,3.155151,3.155762
std,0.362156,0.339627,0.343147
min,3.000000,3.000000,3.000000
25%,3.000000,3.000000,3.000000
50%,3.000000,3.000000,3.000000
75%,3.000000,3.141685,3.000000
max,4.000000,4.000000,4.000000


In [33]:
df_rawdata_copy.describe()[["Totalworkingyears","Totalworkingyears_ITE", "Totalworkingyears_KNN"]]

,Totalworkingyears,Totalworkingyears_ITE,Totalworkingyears_KNN
count,1016.000000,1510.000000,1510.000000
mean,11.314961,11.551796,12.370861
std,7.794100,7.223500,6.803500
min,0.000000,0.000000,0.000000
25%,6.000000,6.970660,8.000000
50%,10.000000,10.000000,12.000000
75%,15.000000,13.973624,16.000000
max,40.000000,40.000000,40.000000


In [34]:
df_rawdata_copy.describe()[["Worklifebalance", "Worklifebalance_ITE", "Worklifebalance_KNN"]]

,Worklifebalance,Worklifebalance_ITE,Worklifebalance_KNN
count,1410.000000,1510.000000,1510.000000
mean,2.756028,2.755562,2.733907
std,0.705888,0.682206,0.692784
min,1.000000,1.000000,1.000000
25%,2.000000,2.000000,2.000000
50%,3.000000,3.000000,3.000000
75%,3.000000,3.000000,3.000000
max,4.000000,4.000000,4.000000


In [35]:
df_rawdata_copy.describe()[["Hourlyrate","Hourlyrate_ITE","Hourlyrate_KNN"]]

,Hourlyrate,Hourlyrate_ITE,Hourlyrate_KNN
count,1430.000000,1510.000000,1510.000000
mean,66.025175,66.025926,65.930728
std,20.270996,19.727124,19.792448
min,30.000000,30.000000,30.000000
25%,48.000000,49.000000,49.000000
50%,66.000000,66.000000,66.000000
75%,84.000000,83.000000,83.000000
max,100.000000,100.000000,100.000000


Dailyrate_KNN, Distancefromhome_KNN, Monthlyincome_KNN, Performancerating_KNN, Totalworkingyears_KNN, Worklifebalance_KNN, Hourlyrate_KNN

# Hemos seleccionado el método KNN Imputer ya que los resultados son más precisos.


In [36]:
#al final me quedo con las del método kn por escoger alguna
df_rawdata_copy.drop(['Dailyrate', 'Dailyrate_ITE', 'Distancefromhome', 'Distancefromhome_ITE', 'Monthlyincome', 'Monthlyincome_ITE', 'Performancerating', 'Performancerating_ITE', 'Totalworkingyears', 'Totalworkingyears_ITE', 'Worklifebalance', 'Worklifebalance_ITE', 'Hourlyrate', 'Hourlyrate_ITE'], axis = 1, inplace = True)

# ahora vamos a cambiar el nombre de las columnas que quedaron para que tengan el mismo nombre de origen
nuevo_nombre = {"Dailyrate_KNN": "Dailyrate",  'Distancefromhome_KNN': "Distancefromhome", "Monthlyincome_KNN": "Monthlyincome", "Performancerating_KNN": "Performancerating","Totalworkingyears_KNN": "Totalworkingyears", "Worklifebalance_KNN": "Worklifebalance", "Hourlyrate_KNN":"Hourlyrate"}
df_rawdata_copy.rename(columns = nuevo_nombre, inplace = True)


In [37]:
df_rawdata_copy.columns

Index(['Age', 'Attrition', 'Businesstravel', 'Department', 'Education',
       'Educationfield', 'Employeenumber', 'Environmentsatisfaction', 'Gender',
       'Jobinvolvement', 'Joblevel', 'Jobrole', 'Jobsatisfaction',
       'Maritalstatus', 'Monthlyrate', 'Numcompaniesworked', 'Overtime',
       'Percentsalaryhike', 'Relationshipsatisfaction', 'Stockoptionlevel',
       'Trainingtimeslastyear', 'Yearsatcompany', 'Yearssincelastpromotion',
       'Yearswithcurrmanager', 'Datebirth', 'Remotework', 'Dailyrate',
       'Distancefromhome', 'Monthlyincome', 'Performancerating',
       'Totalworkingyears', 'Worklifebalance', 'Hourlyrate'],
      dtype='object')

In [38]:
df_rawdata_copy.to_csv("../Data/raw_data_final_20231205.csv")

# Sesión 12 diciembre

In [39]:
df_rawdata_copy= pd.read_csv("../Data/raw_data_final_20231205.csv", index_col=0)

Hemos cambiado los datos numericos de la variable Jobinvolvement por etiquetas

In [40]:
dict_map = {1.0: "Low", 2.0: "Moderate", 3.0: "High", 4.0: "Very High"}
df_rawdata_copy["Jobinvolvement"] = df_rawdata_copy["Jobinvolvement"].replace(dict_map)


In [41]:
df_rawdata_copy["Jobinvolvement"].value_counts()

Jobinvolvement
High         891
Moderate     383
Very High    152
Low           84
Name: count, dtype: int64

### Pasamos a la columna Jobrole

In [42]:
# primero vemos cómo está la columna Jobrole
df_rawdata_copy["Jobrole"].head()

0     Research Director 
1               Manager 
2               Manager 
3     Research Director 
4       Sales Executive 
Name: Jobrole, dtype: object

In [43]:
# capitalize valores de Jobrole (primera letra de cada palabra)
df_rawdata_copy['Jobrole'] = df_rawdata_copy['Jobrole'].apply(lambda x: ' '.join(word.capitalize() for word in x.lower().split()))
df_rawdata_copy["Jobrole"].head()

0    Research Director
1              Manager
2              Manager
3    Research Director
4      Sales Executive
Name: Jobrole, dtype: object

Aplicamos los cambios en Department mediante una función:

In [44]:
def fill_department(df, jobrole, department):
    df.loc[df['Jobrole'] == jobrole, 'Department'] = df.loc[df['Jobrole'] == jobrole, 'Department'].fillna(department)
    return df

# Uso de la función fill_department para reemplazar los valores faltantes en 'Department'
df_rawdata_copy=fill_department(df_rawdata_copy, 'Research Director', 'Research & Development')
df_rawdata_copy=fill_department(df_rawdata_copy, 'Healthcare Representative', 'Research & Development')
df_rawdata_copy=fill_department(df_rawdata_copy, 'Laboratory Technician', 'Research & Development')
df_rawdata_copy=fill_department(df_rawdata_copy, 'Manufacturing Director', 'Research & Development')
df_rawdata_copy=fill_department(df_rawdata_copy, 'Research Scientist', 'Research & Development')
df_rawdata_copy=fill_department(df_rawdata_copy, 'Sales Executive', 'Sales')
df_rawdata_copy=fill_department(df_rawdata_copy, 'Sales Representative', 'Sales')
df_rawdata_copy=fill_department(df_rawdata_copy, 'Human Resources', 'Human Resources')
df_rawdata_copy=fill_department(df_rawdata_copy, 'Manager', 'Unknown')

df_rawdata_copy["Department"] = df_rawdata_copy["Department"].str.strip()
# Mostrar el DataFrame actualizado
df_rawdata_copy[["Jobrole", "Department"]].head(5)

,Jobrole,Department
0,Research Director,Research & Development
1,Manager,Unknown
2,Manager,Research & Development
3,Research Director,Research & Development
4,Sales Executive,Sales


In [45]:
df_rawdata_copy

,Age,Attrition,Businesstravel,Department,Education,Educationfield,Employeenumber,Environmentsatisfaction,Gender,Jobinvolvement,Joblevel,Jobrole,Jobsatisfaction,Maritalstatus,Monthlyrate,Numcompaniesworked,Overtime,Percentsalaryhike,Relationshipsatisfaction,Stockoptionlevel,Trainingtimeslastyear,Yearsatcompany,Yearssincelastpromotion,Yearswithcurrmanager,Datebirth,Remotework,Dailyrate,Distancefromhome,Monthlyincome,Performancerating,Totalworkingyears,Worklifebalance,Hourlyrate
0,51,No,Unknown,Research & Development,3,Unknown,162,Not Satisfied,Male,High,Executive,Research Director,Satisfied,Unknown,6462,7,No,13,Satisfied,0,5,20,15,15,1972,Yes,684.0,6.0,19537.0,3.0,13.6,3.0,51.0
1,52,No,Unknown,Unknown,4,Life Sciences,259,Satisfied,Male,Moderate,Executive,Manager,Satisfied,Unknown,5678,0,Unknown,14,Not Satisfied,1,5,33,11,9,1971,Yes,699.0,1.0,19999.0,3.0,34.0,3.0,65.0
2,42,No,travel_rarely,Research & Development,2,Technical Degree,319,Satisfied,Male,High,Executive,Manager,Very Satisfied,Married,4933,1,No,11,Very Satisfied,0,3,22,11,15,1981,Yes,532.0,4.0,19232.0,3.0,22.0,3.0,58.0
3,47,No,travel_rarely,Research & Development,4,Medical,2069,Not Satisfied,Female,High,Senior,Research Director,Satisfied,Married,26703,3,Unknown,19,Somewhat Satisfied,2,2,20,5,6,1976,No,359.0,2.0,17169.0,3.0,11.2,1.8,82.0
4,46,No,Unknown,Sales,3,Technical Degree,2070,Not Satisfied,Female,Very High,Senior,Sales Executive,Not Satisfied,Divorced,7739,2,No,12,Very Satisfied,1,5,19,2,8,1977,No,1319.0,3.0,4884.6,3.0,13.4,3.0,45.0
5,48,No,Unknown,Research & Development,3,Medical,1900,Very Satisfied,Female,High,Senior,Manager,Very Satisfied,Unknown,2437,3,No,11,Somewhat Satisfied,1,3,22,4,7,1975,Yes,117.0,22.0,17174.0,3.0,14.4,3.0,58.0
6,59,No,Unknown,Sales,3,Life Sciences,81,Not Satisfied,Female,High,Mid-level,Sales Executive,Not Satisfied,Unknown,2354,7,Unknown,11,Very Satisfied,0,3,21,7,9,1964,Yes,1435.0,25.0,10477.4,3.0,28.0,2.0,99.0
7,42,No,travel_rarely,Sales,1,Unknown,387,Somewhat Satisfied,Male,High,Junior,Sales Executive,Satisfied,Married,24532,1,No,25,Satisfied,0,3,20,11,6,1981,No,635.0,1.0,8659.6,4.0,20.0,3.0,99.0
8,41,No,Unknown,Unknown,5,Unknown,2071,Somewhat Satisfied,Female,High,Senior,Manager,Not Satisfied,Married,5626,7,No,16,Somewhat Satisfied,1,2,18,11,8,1982,Yes,1276.0,2.0,16595.0,3.0,22.0,3.0,91.0
9,41,No,travel_frequently,Research & Development,3,Unknown,999,Not Satisfied,Male,High,Executive,Research Director,Satisfied,Unknown,3735,2,No,17,Somewhat Satisfied,1,2,18,0,11,1982,No,840.0,9.0,7971.4,3.0,21.0,4.0,64.0


In [46]:
df_rawdata_copy.to_csv("../Data/raw_data_final_20231212.csv")